# Revisit clustering

In [1]:
# sinteractive --mem=100g --cpus-per-task=16 --time=8:00:00

In [1]:
import os
import sys
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
import dask
from dask import delayed
from dask.distributed import Client, LocalCluster

from fastcluster import linkage

# Project level imports
sys.path.insert(0, '../lib')
from ncbi_remap.config import memory
from ncbi_remap.notebook import Nb
from ncbi_remap.plotting import make_figs

# Connect to data store
store = pd.HDFStore('../output/sra.h5', mode='r')

In [2]:
#cluster = LocalCluster(n_workers=4, threads_per_worker=2, memory_limit='24GB')
#client = Client(cluster)
client = Client()
client

Client Scheduler: tcp://127.0.0.1:38891 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 26.14 GB


In [5]:
def repartition_data():
    samples = store['aln/complete'].srx.unique().tolist()
    store.close()

    fnames = []
    for srx in samples:
        #fnames.append(f'/media/psf/Promise_Pegasus/fearjm/ncbi_remap/output/aln-wf/gene_counts/{srx}.parquet')
        fnames.append(f'../output/aln-wf/gene_counts/{srx}.parquet')
        
    dat = dd.read_parquet(fnames, index=['FBgn', 'srx'])
    dat = dat.repartition(npartitions=2000)
    dat.to_parquet('../output/aln-wf/gene_counts_stack_partition.parquet', write_index=True, compute=True)
    return dat
    
    
def combine_data(dat=None, **kwargs):
    if dat is not None:
        df = dat.compute()
        df.to_parquet('../output/aln-wf/gene_counts_stack.parquet')
        return df
    
    if Path('../output/aln-wf/gene_counts_stack_partition.parquet').exists():
        dat = dd.read_parquet(fname)
        return combine_data(dat=dat)
        
    dat = repartition_data()
    return combine_data(dat=dat)
    
def pivot_data(df=None, **kwargs):
    if df is not None:
        wide = df.unstack()
        wide.columns = wide.columns.droplevel(0)
        wide.to_parquet('../output/aln-wf/gene_counts_wide.parquet')
        return wide
        
    if Path('../output/aln-wf/gene_counts_stack.parquet').exists():
        df = pd.read_parquet('../output/aln-wf/gene_counts_stack.parquet')
        return pivot_data(df=df)
        
    df = combine_data(**kwargs)
    return pivot_data(df=df)
    
    
def calc_corr(wide=None, **kwargs):
    if wide is not None:
        _corr = wide.corr(method='spearman')
        _corr.to_parquet('../output/aln-wf/gene_counts_corr.parquet')
        return _corr
        
    if Path('../output/aln-wf/gene_counts_wide.parquet').exists():
        wide = pd.read_parquet('../output/aln-wf/gene_counts_wide.parquet')
        return calc_corr(wide=wide)
        
    wide = pivot_data(**kwargs)
    return calc_corr(wide=wide)

In [ ]:
if Path('../output/aln-wf/gene_counts_corr.parquet').exists():
    _corr = pd.read_parquet('../output/aln-wf/gene_counts_corr.parquet')
else:
    _corr = calc_corr()

In [ ]:
client.close()

In [17]:
_corr = wide.corr(method='spearman')

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [ ]:
_corr.head()

In [ ]:
_corr.to_parquet('../output/aln-wf/gene_counts_corr.parquet')

In [192]:
link = linkage(_corr, method='average')

In [192]:
g = sns.clustermap(_corr, row_linkage=link, col_linkage=link, xticklabels=False, yticklabels=False)

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
